---
# <div align="center"><font color='green'> COSC 2673/2793 |  Machine Learning | Assignment 1 </font></div>
## <div align="center"> <font color='red'> Student Name:    Arya and Vaibhav                           </font></div>
## <div align="center"> <font color='red'> Student number:                            </font></div>
---

# Problem statement

your problem statement goes here



## Data Splitting, and EDA

 1) Load yur data
 2) Split your Data, explain why and how.
 3) Perform EDA and explain what each step represents. You need to perform EDA with respect to at leats one of the target values: TARGET_Capacity, or	TARGET_CaseCount.
 



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## TODO
## three above steps


### EDA result Analysis and deriven conclusion
Explain what you have observed from the output of your EDA


## Data Pre- Processing

Data preprocessing includes Normalisation or Standarisation, and PCA

In [1]:
## TODO
# You will have a train and test
# then you generate train_S or Train_N for standrarised or/and normalised train, then the same thing for the test
# then you create trian_pca, and test_pca
# you will use all these sets to generate your experiments later on

# Baseline development (Linear regression and Poisson Regression)

Please note that both Linear Regression and Poisson Regression are examples of generalized linear models. Therefore, when you are asked to run linear regression for TARGET_Capacity prediction, it is the same as asking you to select a generalized linear model for your given task. Similarly for Poisson regression.

Note: You also need to balance model complexity with model performance using regularization. Explain how and why.

### Performance metrics selection

Explain the selection of performance metrics for each task in detail, including your justification.

### Parameter tuning

Explain the process of parameter tuning for each task in detail, including your justification.

Note: You also need to balance model complexity with model performance. Explain how and why.

In [ ]:
## TODO
## You need to run this code, so the output is displayed bellow


<span style="font-size:1.5em;">�</span> What observations did you make?

> <span style="font-size:1em;">✔</span> **Observations:** 


## Polynomila regression regression only for TARGET_Capacity prediction

### performance metrics selection
Explain all the above for each task in detail, don't forget to include justification

### Parameter tunning
Explain all the above for each task in detail, don't forget to include justification

Note: You need to also balance the model complexity with model performence, explain how and why

In [ ]:
## TODO
## You need to run this code, so the output is displayed bellow

<span style="font-size:1.5em;">�</span> What observations did you make?

> <span style="font-size:1em;">✔</span> **Observations:** 

### You can add any other analysis you want with justification here (adding code and markdown)